# Merging and Preprocessing

### Load Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import naive_bayes
from datetime import timedelta
import re
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVR
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Player News

In [2]:
news = pd.read_excel('../data/news_data.xlsx')
news.head()

,date,headline,name,news,team
0,2017-12-10 10:40:00,Bucs DT Gerald McCoy fears he suffered a torn ...,Gerald McCoy,\n McCoy has torn his biceps twice...,Buccaneers
1,2017-12-10 09:28:00,Coach Todd Bowles announced Josh McCown suffer...,Josh McCown,\n McCown was banged up in this on...,Jets
2,2017-12-10 09:20:00,Keelan Cole caught all three of his targets fo...,Keelan Cole,\n Cole reeled in a 75-yard touchd...,Jaguars
3,2017-12-10 09:15:00,Dede Westbrook caught 5-of-8 targets for 81 ya...,Dede Westbrook,\n Westbrook's first NFL touchdown...,Jaguars
4,2017-12-10 09:10:00,Marqise Lee caught 5-of-6 targets for 65 yards...,Marqise Lee,\n Lee has turned into a solid flo...,Jaguars


In [3]:
news.tail()

,date,headline,name,news,team
22476,2016-10-20 09:31:00,"Ladarius Green (ankle, PUP) said there is ""no ...",Ladarius Green,"\n Signed to a four-year, $20 mill...",Free Agent
22477,2016-10-20 09:28:00,Gerald McCoy (calf) practiced on a limited bas...,Gerald McCoy,\n It is a step in the right direc...,Buccaneers
22478,2016-10-20 09:25:00,Jordan Reed said he hid the concussion he suff...,Jordan Reed,"\n ""I kind of kept it to myself,"" ...",Redskins
22479,2016-10-20 09:13:00,Colts coach Chuck Pagano said DE Henry Anderso...,Henry Anderson,\n Anderson played through his kne...,Colts
22480,2016-10-20 09:10:00,"With Doug Martin (hamstring) out, Jacquizz Rod...",Jacquizz Rodgers,\n Rodgers was the unquestioned wo...,Buccaneers


### Preprocessing Player News

In [4]:
cities = {
    'Bills':'Buffalo',
    'Dolphins':'Miami',
    'Patriots':'New England',
    'Jets':'New York',
    'Ravens':'Baltimore',
    'Bengals':'Cincinnati',
    'Browns':'Cleveland',
    'Steelers':'Pittsburgh',
    'Texans':'Houston',
    'Colts':'Indianapolis',
    'Jaguars':'Jacksonville',
    'Titans':'Tennessee',
    'Broncos':'Denver',
    'Chiefs':'Kansas City',
    'Chargers':'Los Angeles',
    'Raiders':'Oakland',
    'Cowboys':'Dallas',
    'Giants':'New York',
    'Eagles':'Philadelphia',
    'Redskins':'Washington',
    'Bears':'Chicago',
    'Lions':'Detroit',
    'Packers':'Green Bay',
    'Vikings':'Minnesota',
    'Falcons':'Atlanta',
    'Panthers':'Carolina',
    'Saints':'New Orleans',
    'Buccaneers':'Tampa Bay',
    'Cardinals':'Arizona',
    'Rams':'Los Angeles',
    '49ers':'San Francisco',
    'Seahawks':'Seattle'  
}

default_cities = defaultdict(str)
for city in cities.keys():
    default_cities[city] = cities[city]

In [5]:
regex = re.compile('[^a-zA-Z]')
to_display = ['name','news','news_clean','news_unigrams','news_bigrams']
stopWords = set(stopwords.words('english'))

def process_text(row):
    '''
    if the news mentions that player -> self
    if the news mentions the player's team -> ownteam
    if the news mentions the player's city -> owncity
    '''
    parsed_sentence = str()
    names = row['name'].split()
    for word in row['news'].split():
        word = regex.sub('',word)
        if word in names:
            parsed_sentence = parsed_sentence+'self '
        elif word ==row['team']:
            parsed_sentence = parsed_sentence+'ownteam '
        elif word in default_cities[row['team']].split():
            parsed_sentence = parsed_sentence+'owncity '
        elif word in stopWords:
            pass
        else:
            parsed_sentence = parsed_sentence+word+' '
    return parsed_sentence

def get_bigrams(myString):
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(myString)
    stemmer = PorterStemmer()
    bigram_finder = BigramCollocationFinder.from_words(tokens)
    bigram_tuples = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 500)
    bigrams = []
    for bigram_tuple in bigram_tuples:
        x = "%s %s" % bigram_tuple
        bigrams.append(x)
        
    result = [' '.join([stemmer.stem(w).lower() for w in x.split()]) for x in bigrams]
    return result

def get_unigrams(myString):
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(myString)
    stemmer = PorterStemmer()
    result = [stemmer.stem(w).lower() for w in tokens if w.lower() not in stopWords]
    return result

news['news_clean'] = news.apply(process_text, axis = 1)
news['news_unigrams'] = news['news_clean'].apply(lambda x: get_unigrams(x))
news['news_bigrams'] = news['news_clean'].apply(lambda x: get_bigrams(x))
news[to_display].head()

,name,news,news_clean,news_unigrams,news_bigrams
0,Gerald McCoy,\n McCoy has torn his biceps twice...,self torn biceps twice already pro career tear...,"[self, torn, bicep, twice, alreadi, pro, caree...","[falcon panther, he know, panther saint, saint..."
1,Josh McCown,\n McCown was banged up in this on...,self banged one injuring back second quarter h...,"[self, bang, one, injur, back, second, quarter...","[bryce petti, hell undergo, it tough, mccown g..."
2,Keelan Cole,\n Cole reeled in a 75-yard touchd...,self reeled yard touchdown beautiful catch thr...,"[self, reel, yard, touchdown, beauti, catch, t...","[blake bortl, bortl give, it second, wr head, ..."
3,Dede Westbrook,\n Westbrook's first NFL touchdown...,Westbrooks first NFL touchdown came yard corne...,"[westbrook, first, nfl, touchdown, came, yard,...","[hi eight, nfl touchdown, texan defens, came y..."
4,Marqise Lee,\n Lee has turned into a solid flo...,self turned solid floor play fantasy five catc...,"[self, turn, solid, floor, play, fantasi, five...","[after catch, wr option, week meet, with week,..."


### Player Scores and Averages

In [6]:
#Match up scores with articles
scores = pd.read_csv('../data/fantasy_points_data.csv', parse_dates= True)
scores.head()

,Player,Team,Position,Year,Week,Points
0,Drew Brees,NO,QB,2016,1,37.7
1,Andrew Luck,IND,QB,2016,1,37.4
2,Alex Smith,KC,QB,2016,1,33.7
3,Jameis Winston,TB,QB,2016,1,30.4
4,Matthew Stafford,DET,QB,2016,1,29.5


In [7]:
def yearly_average(row):
    if row['Year']==2016:
        return row['Points']/weeks_in_2016
    elif row['Year']==2017:
        return row['Points']/weeks_in_2017

In [8]:
#Computing average over all weeks in year
weeks_in_2016 = scores[scores.Year==2016]['Week'].max()
weeks_in_2017 = scores[scores.Year==2017]['Week'].max()
avg_scores = scores.groupby(['Player','Year'], as_index=False)['Points'].sum()
avg_scores['Avg_Points']= avg_scores.apply(yearly_average,axis=1)
avg_scores.columns = ['Player', 'Year', 'SeasonPoints', 'AvgSeasonPoints']

In [9]:
#Computing averages over all weeks *played* in a year
wkavg_scores = scores.groupby(['Player', 'Year'], as_index=False)['Points'].mean()
wkavg_scores.columns = ['Player', 'Year', 'AvgWkPoints']
wkavg_scores.head()

,Player,Year,AvgWkPoints
0,A.J. Derby,2016,3.425000
1,A.J. Derby,2017,4.050000
2,A.J. Green,2016,13.377778
3,A.J. Green,2017,10.692308
4,AJ McCarron,2017,1.900000


In [10]:
scores_combined = scores.merge(avg_scores, 
                               how='inner', 
                               on=['Player','Year'])
scores_combined = scores_combined.merge(wkavg_scores,
                                  how='inner',
                                  on=['Player', 'Year'])
scores_combined['Diff_from_Avg'] = scores_combined['Points'] - scores_combined['AvgSeasonPoints']
scores_combined['Diff_from_WkAvg'] = scores_combined['Points'] - scores_combined['AvgWkPoints'] 
scores_combined.head()

,Player,Team,Position,Year,Week,Points,SeasonPoints,AvgSeasonPoints,AvgWkPoints,Diff_from_Avg,Diff_from_WkAvg
0,Drew Brees,NO,QB,2016,1,37.7,338.5,19.911765,21.15625,17.788235,16.54375
1,Drew Brees,NO,QB,2016,2,14.5,338.5,19.911765,21.15625,-5.411765,-6.65625
2,Drew Brees,NO,QB,2016,3,25.9,338.5,19.911765,21.15625,5.988235,4.74375
3,Drew Brees,NO,QB,2016,4,11.8,338.5,19.911765,21.15625,-8.111765,-9.35625
4,Drew Brees,NO,QB,2016,6,32.6,338.5,19.911765,21.15625,12.688235,11.44375


### Merging

In [11]:
start_of_season = {2016:pd.datetime.strptime('2016-09-08', '%Y-%m-%d'),
                  2017:pd.datetime.strptime('2017-09-08', '%Y-%m-%d')}

In [12]:
#Add in the first and last date for a given playing week
scores_combined['max_date'] = scores_combined.apply(lambda x: start_of_season[x['Year']] + timedelta(weeks = x['Week']), axis=1)
scores_combined['min_date'] = scores_combined.apply(lambda x: start_of_season[x['Year']] + timedelta(weeks = x['Week'] - 1), axis=1)

In [13]:
scores_combined.dtypes

Player                     object
Team                       object
Position                   object
Year                        int64
Week                        int64
Points                    float64
SeasonPoints              float64
AvgSeasonPoints           float64
AvgWkPoints               float64
Diff_from_Avg             float64
Diff_from_WkAvg           float64
max_date           datetime64[ns]
min_date           datetime64[ns]
dtype: object

In [14]:
news.dtypes

date             datetime64[ns]
headline                 object
name                     object
news                     object
team                     object
news_clean               object
news_unigrams            object
news_bigrams             object
dtype: object

In [15]:
#Merge news with scores and filter out scores that are not within a playing week
combined = scores_combined.merge(news, how='inner', left_on='Player', right_on='name')
combined = combined[(combined['date'] <= combined['max_date']) & 
                    (combined['date'] >= combined['min_date'])]

In [16]:
combined.shape

(11105, 21)

In [26]:
mismatched_scores = scores_combined.merge(news, how='left', left_on='Player', right_on='name')
mismatched_scores = mismatched_scores[(mismatched_scores['date'] <= mismatched_scores['max_date']) 
                                      & (mismatched_scores['date'] >= mismatched_scores['min_date']) 
                                      #& (mismatched_scores['name'] == np.nan)]
                                     ]
mismatched_news = scores_combined.merge(news, how='right', left_on='Player', right_on='name')
#mismatched_news = mismatched_news[mismatched_news['Player'] == np.nan]

In [46]:
#Number of players with no scores
len(news[np.invert(np.in1d(news.name, scores_combined.Player))].name.value_counts())

1390

In [47]:
#Count of news articles with no scores
np.sum(news[np.invert(np.in1d(news.name, scores_combined.Player))].name.value_counts())

6387

In [48]:
news[np.invert(np.in1d(news.name, scores_combined.Player))].name.value_counts()

Odell Beckham         88
Luke Kuechly          49
Steve Smith Sr.       43
Tyron Smith           40
Vontae Davis          39
John Ross             39
Ryan Kalil            37
Terron Armstead       35
Ted Ginn              33
Darrelle Revis        32
Jimmy Smith           32
Trent Williams        29
Cordy Glenn           29
Joe Haden             28
Muhammad Wilkerson    28
Justin Houston        28
David Amerson         28
Tyrann Mathieu        27
Teddy Bridgewater     27
Michael Bennett       27
Mike Pouncey          27
Dont'a Hightower      27
Janoris Jenkins       26
Kyle Shanahan         26
Ezekiel Ansah         26
Jason Pierre-Paul     26
Clay Matthews         25
Jalen Ramsey          25
DeAndre Levy          25
Earl Thomas           25
                      ..
Morgan Fox             1
Cameron Stingily       1
Jude Adjei-Barimah     1
Tenny Palepoi          1
Corbin Bryant          1
Matt Wile              1
Alexander Armah        1
Glenn Gronkowski       1
Nate Sudfeld           1


In [22]:
mismatched_scores.head()

,Player,Team,Position,Year,Week,Points,SeasonPoints,AvgSeasonPoints,AvgWkPoints,Diff_from_Avg,...,max_date,min_date,date,headline,name,news,team,news_clean,news_unigrams,news_bigrams
20,Drew Brees,NO,QB,2016,1,37.7,338.5,19.911765,21.15625,17.788235,...,2016-09-15,2016-09-08,2016-09-11 04:33:00,Drew Brees completed 28-of-42 passes for 424 y...,Drew Brees,\n Aside from an early lost fumble...,Saints,Aside early lost fumble self played quarterbac...,"[asid, earli, lost, fumbl, self, play, quarter...","[asid earli, the highlight, bumbl defens, defe..."
21,Drew Brees,NO,QB,2016,1,37.7,338.5,19.911765,21.15625,17.788235,...,2016-09-15,2016-09-08,2016-09-08 11:56:00,Drew Brees got a $30 million signing bonus on ...,Drew Brees,\n That bonus will be spread out a...,Saints,That bonus spread across next five years ownte...,"[bonu, spread, across, next, five, year, ownte...","[that bonu, the entir, across next, base salar..."
57,Drew Brees,NO,QB,2016,2,14.5,338.5,19.911765,21.15625,-5.411765,...,2016-09-22,2016-09-15,2016-09-18 05:32:00,Drew Brees completed 29-of-44 passes for 263 y...,Drew Brees,\n Brees got off to a very slow st...,Saints,self got slow start averaging five yards per a...,"[self, got, slow, start, averag, five, yard, p...","[atlanta next, giant put, he didnt, it quiet, ..."
94,Drew Brees,NO,QB,2016,3,25.9,338.5,19.911765,21.15625,5.988235,...,2016-09-29,2016-09-22,2016-09-27 12:01:00,Drew Brees completed 36-of-54 passes for 376 y...,Drew Brees,"\n Unfortunately for the Saints, t...",Saints,Unfortunately ownteam defense couldnt stop Fal...,"[unfortun, ownteam, defens, couldnt, stop, fal...","[cobi fleener, diego week, fleener two, ingram..."
131,Drew Brees,NO,QB,2016,4,11.8,338.5,19.911765,21.15625,-8.111765,...,2016-10-06,2016-09-29,2016-10-02 08:25:00,Drew Brees completed 23-of-36 passes for 206 y...,Drew Brees,\n It was not a vintage performanc...,Saints,It vintage performance self He struggled accur...,"[vintag, perform, self, struggl, accuraci, gam...","[charger gift, it vintag, panther home, qb fac..."


### Output

In [49]:
#combined.to_csv('../data/news_and_scores.csv', index=False)
combined.to_excel('../data/news_and_scores.xlsx', index=False)

In [18]:
mismatched_scores.to_csv('../data/mismatched_scores.csv', index=False)
mismatched_news.to_csv('../data/mismatched_news.csv')

In [19]:
mismatched_scores.shape

(0, 21)

In [20]:
mismatched_news.shape

(0, 21)